### 实施 FunkSVD

在此 notebook 中，我们将自己编写实施 FunkSVD 的函数，它会遵守你在上个视频中看到的步骤。如果你认为你还没有准备好自己完成这个任务，可以跳到下个视频，观看我是如何完成这些步骤的。

我们将用之前使用的数据子集检测我们的算法。首先请运行以下单元格。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` 你将使用 **user_movie_subset** 矩阵演示你的 FunkSVD 算法将收敛。在以下单元格中，请参考注释和文档字符串编写你自己的 FunkSVD 函数。你也可以自己完成这些函数，不参考注释部分。你可以随意增减函数代码，使你的代码能正常运行。 

**注意：**此版矩阵分解没有 ∑ 矩阵。

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` 请在 **user_movie_subset** 数据集上尝试你的函数。先将潜在特征设为 4 个，学习速率设为 0.005，并迭代 10 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.886217
2 		 14.119104
3 		 3.853599
4 		 2.742664
5 		 2.620110
6 		 2.565105
7 		 2.519750
8 		 2.473837
9 		 2.424408
10 		 2.370333


In [4]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.11493288  8.87309589 10.68374239 10.05780963]
 [ 8.47030944  7.42351755  8.68483848  9.22026831]
 [ 8.15614235  7.08761926  8.65856459  7.19480756]
 [ 9.74447075  8.43589993 10.13978335  9.27940914]
 [ 8.74829619  7.48567163  8.93007565  8.38702897]
 [ 6.96405405  6.21090742  7.63428059  6.31349971]
 [ 9.45293671  8.14545608  9.76126995  8.90595054]
 [ 8.03827636  7.11297135  8.53676306  8.37332197]
 [ 8.0361171   7.4433943   9.10187108  8.55228598]
 [ 7.84428371  6.90215571  8.45728293  7.12172523]
 [ 9.11159572  7.8579801   9.38300572  8.59483251]
 [ 9.46713751  8.42016674 10.14625074  9.52323577]
 [ 9.59340516  8.23472241  9.96786759  8.74579851]
 [ 6.0849538   5.56967313  6.53974347  7.43322716]
 [ 9.63906175  8.52224301 10.18849952  9.81765856]
 [ 9.73262934  8.39677704  9.84835976 10.13451216]
 [ 8.35171153  7.1227631   8.29540054  8.67489324]
 [ 7.18453976  6.06682564  6.94019793  7.49997573]
 [ 7.74771001  7.2874458   8.83724734  8.74608555]
 [ 8.60239769  7.1837033   8.41

**请在此处填写你的结论。**

`3.` 再在 **user_movie_subset** 数据集上尝试你的函数。这次潜在特征为 4 个，学习速率设为 0.005。但是，迭代次数提高到 250 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？迭代 250 次后，误差会怎样？

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 47.334184
2 		 17.774658
3 		 4.596992
4 		 2.834490
5 		 2.687812
6 		 2.642430
7 		 2.605104
8 		 2.565927
9 		 2.522480
10 		 2.473693
11 		 2.418745
12 		 2.356880
13 		 2.287414
14 		 2.209798
15 		 2.123692
16 		 2.029054
17 		 1.926232
18 		 1.816051
19 		 1.699871
20 		 1.579599
21 		 1.457633
22 		 1.336716
23 		 1.219717
24 		 1.109356
25 		 1.007926
26 		 0.917076
27 		 0.837685
28 		 0.769867
29 		 0.713079
30 		 0.666298
31 		 0.628228
32 		 0.597472
33 		 0.572677
34 		 0.552618
35 		 0.536237
36 		 0.522658
37 		 0.511169
38 		 0.501205
39 		 0.492321
40 		 0.484166
41 		 0.476464
42 		 0.468997
43 		 0.461591
44 		 0.454102
45 		 0.446412
46 		 0.438425
47 		 0.430057
48 		 0.421239
49 		 0.411910
50 		 0.402024
51 		 0.391540
52 		 0.380434
53 		 0.368687
54 		 0.356296
55 		 0.343272
56 		 0.329638
57 		 0.315434
58 		 0.300713
59 		 0.285547
60 		 0.270020
61 		 0.254230
62 		 0.238290
63 		 0.222316
64 		

In [6]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99998804  9.99999477  9.99999697 10.00001176]
 [10.00000264  4.00000113  9.00000071  9.99999759]
 [ 7.99999534  8.99999807  9.99999895  5.00000468]
 [ 8.99999295  7.99999708  9.9999984  10.00000702]
 [ 9.99998462  4.99999349  8.99999633  9.0000152 ]
 [ 5.99999189  3.99999676  9.99999828  6.00000811]
 [ 8.99999415  7.99999801  9.9999991   9.00000591]
 [10.00001498  5.00000694  9.00000426  7.99998558]
 [ 7.00000653  8.00000317 10.00000201  7.99999377]
 [ 9.00001456  5.00000635  9.00000377  6.99998595]
 [ 8.99999079  7.99999605  9.99999776  8.00000912]
 [ 9.00000167 10.00000097 10.00000069  8.99999849]
 [10.000009    9.00000399 10.00000239  7.99999134]
 [ 5.00001862  8.00000816  5.00000477  7.99998181]
 [ 9.99999662  7.99999824  9.99999891 10.0000034 ]
 [ 8.99999709  8.99999854  9.99999911 10.00000292]
 [ 8.99998814  7.99999477  7.99999695  8.00001166]
 [ 9.99999919  7.99999974  0.99999987 10.00000076]
 [ 4.99999853  5.99999939  9.99999969 10.00000153]
 [ 8.00001081  7.00000468 10.00

**请在此处填写你的结论。**

上次在此矩阵里放入 **np.nan** 值时，python 中的整个 SVD 算法崩溃了。我们看看使用你的 FunkSVD 函数是否依然会这样。在以下单元格中，我在你的 numpy 数组的第一个单元格中放入了一个 NaN 值。  

`4.` 假设潜在特征有 4 个，学习速率为 0.005，并且迭代 250 次。你能够运行你的 SVD 并且不崩溃吗（内置的 python 方法会崩溃）？你获得了 NaN 值的预测值吗？缺失值的预测是多少？请在以下单元格中回答这些问题。

In [7]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 42.639919
2 		 14.168215
3 		 3.960705
4 		 2.833399
5 		 2.715572
6 		 2.663064
7 		 2.618003
8 		 2.571462
9 		 2.520588
10 		 2.464105
11 		 2.401324
12 		 2.331948
13 		 2.256079
14 		 2.174265
15 		 2.087527
16 		 1.997346
17 		 1.905577
18 		 1.814292
19 		 1.725574
20 		 1.641276
21 		 1.562827
22 		 1.491089
23 		 1.426323
24 		 1.368246
25 		 1.316161
26 		 1.269113
27 		 1.226045
28 		 1.185929
29 		 1.147848
30 		 1.111049
31 		 1.074955
32 		 1.039166
33 		 1.003442
34 		 0.967674
35 		 0.931865
36 		 0.896096
37 		 0.860505
38 		 0.825259
39 		 0.790531
40 		 0.756484
41 		 0.723251
42 		 0.690926
43 		 0.659561
44 		 0.629160
45 		 0.599687
46 		 0.571071
47 		 0.543216
48 		 0.516014
49 		 0.489353
50 		 0.463130
51 		 0.437261
52 		 0.411685
53 		 0.386371
54 		 0.361323
55 		 0.336573
56 		 0.312189
57 		 0.288261
58 		 0.264901
59 		 0.242235
60 		 0.220393
61 		 0.199503
62 		 0.179682
63 		 0.161032
64 		

In [9]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 10.728553974785756:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


下面扩展到更真实的例子上。遗憾的是，依然不建议在本地机器上对整个用户-电影矩阵运行此函数。但是，我们能够查看这个示例扩展到 1000 个用户的情况。在上述部分，你使用了一个非常小的数据子集，并且没有缺失值。

`5.` 考虑到此矩阵的大小，运行时间会比较长。假设超参数如下所示：潜在特征为 4 个，学习速率为0.005，迭代次数为20。运行时间会很久，吃点东西，散散步吧。

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.648060
2 		 10.817988


KeyboardInterrupt: 

`6.` 现在你已经获得了每个用户-电影对的预测值，请根据你的结果回答几个问题。请在以下每个变量对应的括号里填写正确的值，并使用以下测试检测你的答案。

In [11]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [ ]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

```python

```